### **Import libraries**
`lab_v2` is a library containing methods that are often used throughout our GPT experiments.  
This library can be found at the following url:

In [97]:
from typing import List
from collections import Counter

import pandas
from lab_v2.gpt_eval.eval import Eval
from lab_v2.io.pandas import PandasIO
from lab_v2.statistics.entropy import ShannonEntropy

### **Constants**
Specify a few constants to make the notebook easier to configure in the future.

In [98]:
RESPONSE_EXTRACTION = Eval.extract_last_letters

# These must absolutely be changed from test to test
QUESTION_SET_FILE_PATH = 'data/question-set/last_letters.jsonl'
QUESTION_SET_INDEX_NAME = 'iIndex'
QUESTION_SET_ANSWER_NAME = 'answer'

RESPONSE_FILE_PATH = 'data/responses/last_letters/sample_0.jsonl'
RESPONSE_INDEX_NAME = 'question_id'
RESPONSE_SAMPLE_NAME = 'choices'

EXTRACT_RESPONSE = lambda response: response['message']['content']

# These are here just to make it easier
RESPONSE_ANSWERS_NAME = 'answers'

MAJORITY_ANSWER_NAME = 'majority_answer'
MAJORITY_CORRECT_NAME = 'majority_correct'

ENTROPY_COLUMN = 'shannon_entropy'



### **File I/O**
Specify a few functions that make it easier to read from files using pandas

In [99]:
question_set = PandasIO.read_file(QUESTION_SET_FILE_PATH)
question_set = question_set[[QUESTION_SET_INDEX_NAME, QUESTION_SET_ANSWER_NAME]]
display(len(question_set))
question_set.head(1)

3000

,iIndex,answer
0,0,rany


In [100]:
responses = PandasIO.read_file(RESPONSE_FILE_PATH)
display(len(responses))
responses.head(1)

3000

,question_id,id,object,created,model,choices,usage,question,n,temperature
0,31,chatcmpl-7XlFtWDh2tBIfBANVqU5WPrpi3cj6,chat.completion,1688280829,gpt-3.5-turbo,"[{'index': 0, 'message': {'role': 'assistant',...","{'prompt_tokens': 55, 'completion_tokens': 149...","At the end, say 'the answer is [put the concat...",20,0.7


In [101]:
joined = responses.set_index(RESPONSE_INDEX_NAME).join(question_set.set_index(QUESTION_SET_INDEX_NAME))
display(len(joined))
joined.head(1)

3000

,id,object,created,model,choices,usage,question,n,temperature,answer
question_id,,,,,,,,,,
31,chatcmpl-7XlFtWDh2tBIfBANVqU5WPrpi3cj6,chat.completion,1688280829,gpt-3.5-turbo,"[{'index': 0, 'message': {'role': 'assistant',...","{'prompt_tokens': 55, 'completion_tokens': 149...","At the end, say 'the answer is [put the concat...",20,0.7,yral


In [102]:
def extract_answers(row, column):
    answers = []
    for _, element in enumerate(row[column]): 
        response = EXTRACT_RESPONSE(element)
        answers.append(RESPONSE_EXTRACTION(response))
    row[RESPONSE_ANSWERS_NAME] = answers
    return row

joined = joined.apply(lambda row: extract_answers(row, RESPONSE_SAMPLE_NAME), axis=1)
joined.head(1)

,id,object,created,model,choices,usage,question,n,temperature,answer,answers
question_id,,,,,,,,,,,
31,chatcmpl-7XlFtWDh2tBIfBANVqU5WPrpi3cj6,chat.completion,1688280829,gpt-3.5-turbo,"[{'index': 0, 'message': {'role': 'assistant',...","{'prompt_tokens': 55, 'completion_tokens': 149...","At the end, say 'the answer is [put the concat...",20,0.7,yral,"[yrela, yagl, dyal, yrela, yagl, yagl, yal, ya..."


In [103]:
def get_majority(answers: List[str]):
    counter = Counter(answers)
    return counter.most_common()[0][0]
joined[MAJORITY_ANSWER_NAME] = joined[RESPONSE_ANSWERS_NAME].apply(lambda row : get_majority(row))
joined[MAJORITY_CORRECT_NAME] = joined.apply(lambda row : row[MAJORITY_ANSWER_NAME] == row[QUESTION_SET_ANSWER_NAME], axis=1)
joined.head(1)

,id,object,created,model,choices,usage,question,n,temperature,answer,answers,majority_answer,majority_correct
question_id,,,,,,,,,,,,,
31,chatcmpl-7XlFtWDh2tBIfBANVqU5WPrpi3cj6,chat.completion,1688280829,gpt-3.5-turbo,"[{'index': 0, 'message': {'role': 'assistant',...","{'prompt_tokens': 55, 'completion_tokens': 149...","At the end, say 'the answer is [put the concat...",20,0.7,yral,"[yrela, yagl, dyal, yrela, yagl, yagl, yal, ya...",yagl,False


In [104]:
joined[ENTROPY_COLUMN] = joined[RESPONSE_ANSWERS_NAME].apply(lambda row : ShannonEntropy.base(row))
joined.head(1)

,id,object,created,model,choices,usage,question,n,temperature,answer,answers,majority_answer,majority_correct,shannon_entropy
question_id,,,,,,,,,,,,,,
31,chatcmpl-7XlFtWDh2tBIfBANVqU5WPrpi3cj6,chat.completion,1688280829,gpt-3.5-turbo,"[{'index': 0, 'message': {'role': 'assistant',...","{'prompt_tokens': 55, 'completion_tokens': 149...","At the end, say 'the answer is [put the concat...",20,0.7,yral,"[yrela, yagl, dyal, yrela, yagl, yagl, yal, ya...",yagl,False,2.121699
